In [1]:
import sys
import shutil
import os
import operator

import time

from typing import List, Dict, Union, Any, Callable
from collections import Counter, defaultdict,OrderedDict
from dataclasses import dataclass

import import_ipynb
sys.path.append('../')  # Go up two folders to the project root
import building_data_structures.Lexicon as Lexicon

importing Jupyter notebook from C:\Users\gabri\Documents\GitHub\MultimediaProject\building_data_structures\..\building_data_structures\Lexicon.ipynb


In [2]:
# Since this is a simple data class, intializing it can be abstracted with
# the use of dataclass decorator.
# https://docs.python.org/3/library/dataclasses.html

@dataclass
class Posting:
    doc_id: int
    payload: Any = None
    
    @classmethod 
    def from_string(cls, description:str):
        docId,payl=description.split(":")
        doc_id=int(docId)
        payload=int(payl)
        return cls(doc_id, payload)
               
class InvertedIndex:

    def __init__(self):
        self._index = defaultdict(list)
        
    def add_posting(self, term: str, doc_id: int, payload: Any=None) -> None:
        """Adds a document to the posting list of a term."""
        # append new posting to the posting list
        if (self.get_postings(term)==None):
            self._index[term]=[]
        self._index[term].append(Posting(doc_id,payload))
             
    def get_postings(self, term: str) -> List[Posting]:
        """Fetches the posting list for a given term."""
        if (term in self._index):
            return self._index[term]
        return None
    
    def write_to_block(self,file_name_index: str) -> None:
        """ Write the inverted index in lexicographical oreder into a file_name_index on disk."""
        sorted_lexicon=sorted(self._index.items())
        with open(file_name_index, "w") as f:
            for term,postings in sorted_lexicon:
                f.write(term)
                for posting in postings:
                    f.write(f" {posting.doc_id}")
                    if posting.payload:
                        f.write(f":{str(posting.payload)}")
                f.write("\n")
    
    def is_empty(self)->bool:
        """Check if there is no term in the inverted index."""
        return len(self.get_terms())==0
    
    def get_terms(self) -> List[str]:
        """Returns all unique terms in the index."""
        return self._index.keys() 
    
    def clear_structure(self):
        """ It clears the inverted index data structure."""
        self._index.clear()
    
    def get_structure(self):
        """Returns the inverted index data structure."""
        return self._index

In [3]:
class IndexBuilder:
    
    MAX_CHARATER=chr(1114111)
    
    OUTPUT_FILE_FORMAT=".txt"
    NAME_DOC_IDS_FILE="docIds"
    NAME_TERM_FREQ_FILE="termFreq"
    
    def __init__(self):
        print ("Index Builder costructor")
        #self._invertedIndex = InvertedIndex()

    def build_in_memory_index(self,list_of_documents:list)->InvertedIndex:
        """Given a list of document, build an Inverted Index in main Memory (RAM) and return it."""
        invertedIndex = InvertedIndex()
        for doc in list_of_documents:
            doc_id=list_of_documents.index(doc)
            tc = Counter(doc.lower().split())  # dict with term counts, QUI USARE DIRETTAMENTE IL CONTENUTO GIA' PRE-PROCESSATO
            for term, freq in tc.items():
                invertedIndex.add_posting(term, doc_id, freq)
        return invertedIndex


    def build_block_sort_base_indexing(self,list_of_documents:list,output_file_name:str,block_size: int=2200,split_results_in_files:bool=False, delete_intermediete_results:bool=True)-> None:
        """ Given a list of document, build an Inverted Index exploiting both Memory(RAM) at blocks and Disk. 
            It saves the entire structure on disk at location output_file_name.
            The file structure is like term doc_id:term_freq
               
         Args:
            list_of_documents: List of document to be processed.
            output_file_name: The location of where the structure is saved.
            block_size: The size of the block to elaborate in main memory and store on disk as intermediate result.
            split_results_in_files: Specify if you want or not the inderted index in two different files: the doc_ids file and the term_freq file
            delete_intermediete_results: Flag to remove partial results at the end of the procedure or not.
         
        """

        ind = InvertedIndex()
        document_index = Lexicon.DocumentIndex()
        nr_block=0

        DIR_FOLDER="TEMP"

        if os.path.exists(DIR_FOLDER):
            shutil.rmtree(DIR_FOLDER)

        os.makedirs(DIR_FOLDER)

    
        if (split_results_in_files):
            if os.path.exists(output_file_name+"_"+self.NAME_DOC_IDS_FILE+self.OUTPUT_FILE_FORMAT):
                os.remove(output_file_name+"_"+self.NAME_DOC_IDS_FILE+self.OUTPUT_FILE_FORMAT)
            
            if os.path.exists(output_file_name+"_"+self.NAME_TERM_FREQ_FILE+self.OUTPUT_FILE_FORMAT):
                os.remove(output_file_name+"_"+self.NAME_TERM_FREQ_FILE+self.OUTPUT_FILE_FORMAT)
        else:
            if os.path.exists(output_file_name+self.OUTPUT_FILE_FORMAT):
                os.remove(output_file_name+self.OUTPUT_FILE_FORMAT)
    
        #Map phase - read all the documents and write the index at blocks on disk when memory is full, cleaning the memory data structure.

        Lexicon.create_folder("Document_index")
        
        for doc in list_of_documents:
            # Divido il doc_id dal contenuto del documento vero e proprio
            doc_list = doc.split()
            doc_id = int(doc_list[0])
            text = ' '.join(doc_list[1:])

            if (sys.getsizeof(document_index.get_structure()) > 2200):  
                Lexicon.write_to_block("Document_index/document_index.txt", document_index.get_structure())
                document_index.clear_structure()

            document_index.add_document(doc_id, text)

            tc = Counter(text.lower().split())  # dict with term counts, QUI USARE DIRETTAMENTE IL CONTENUTO GIA' PRE-PROCESSATO
            for term, freq in tc.items():
                if (sys.getsizeof(ind.get_structure()) > block_size):  #Free memory available
                    ind.write_to_block(DIR_FOLDER+"/inv_index_"+str(nr_block)+".txt")
                    ind.clear_structure()
                    nr_block=nr_block+1 
                ind.add_posting(term, doc_id, freq)
            
        if (not document_index.is_empty()):   
            Lexicon.write_to_block("Document_index/document_index.txt", document_index.get_structure())
                
        #Finally, saving the last remaing block.       
        if (not ind.is_empty()):   
            ind.write_to_block(DIR_FOLDER+"/inv_index_"+str(nr_block)+".txt")


        # ----  Second part ----

        #Reduce phase --> merging all the blocks (multi-way merge sort) in one unique result and save it on disk. 
                       
        try:
            file_paths = [DIR_FOLDER+"/"+f for f in os.listdir(DIR_FOLDER)] 
            input_files = [open(file, 'r') for file in file_paths]  #Open all the blocks in parallel
            lines=[file.readline().strip() for file in input_files] #Read the first line of each block

            while (not self.__check_all_blocks_are_read(lines)):

                terms=[line.split()[0] if line else self.MAX_CHARATER for line in lines] #Contains a list of strings: the terms present at each line read in the blocks, if file is over put the max-unicode charater
                postings=[" ".join(line.split()[1:]) for line in lines] #Contains a list of strings: the postings present at each line read in the blocks.

                min_term=min(terms)  #Enstablishing what is the term to be considered: the term lexicografical ordered.

                #Merging the postings with the same min term
                mergePostings=[]
                for i in range (0,len(postings)):
                    if (min_term==terms[i]):  
                        self.__decode_string_posting_list_and_merge_to_current(mergePostings,postings[i].split())

                #Sorting by doc_id   
                mergePostings=sorted(mergePostings, key=operator.attrgetter('doc_id'))    
                self.__write_term_posting_list_to_disk(output_file_name,min_term,mergePostings,split_results_in_files)

                #Advance on reading the files in parallel only for blocks where was present the last min term.
                lines=[input_files[i].readline().strip() if (min_term==terms[i]) else lines[i] for i in range (0,len(terms))] 
        except Exception as e:   
            raise e
        finally:
            #Be sure to close all the opened files in parallel
            for file in input_files:
                file.close()  

        if (delete_intermediete_results):
            shutil.rmtree(DIR_FOLDER)

        Lexicon.create_lexicon("complete_inverted_index.txt", "lexicon", "Lexicon/", ".txt", 2200, document_index)   

    
    #Private methods: all utilities used inside main methods to make code more readable.
    
    def __check_all_blocks_are_read(self,lines:list)->bool:
        """ This method is used to check whether all files opened in parallel have been completely read or not
            It checks the contents that have been read to determine the condition.
            
            Args:
                 lines: A list of lines(str) read before.
        """
        for line in lines:
            if line:
                return False
        return True
    
    
    def __decode_string_posting_list_and_merge_to_current(self,current_postings_list:list, new_string_posting_list:str):
        """ This method is used for adding/merging a posting list in string format to the list of current_postings. 
            This method check if current_postings already contains a posting with a doc_id, in that case the term_freq is sum
            otherwise the posting is append to the current_postings.
            
            Args:
                current_postings_list: the actual posting list
                new_string_posting_list: the new posting list to be merged in string format ex. doc_id_1:term_freq_1 doc_id_2:term_freq_2  
        """
        current_docIds=[] #Used to store and retrieve rapidly the actual doc_id in the current_postings_list
        if (len(current_postings_list)>0):
            current_docIds=[curr_posting.doc_id for curr_posting in current_postings_list]
        
        for posting_str in new_string_posting_list: #posting_str is a single posting in the form "docId:freq"
            posting=Posting.from_string(posting_str)
            if (posting.doc_id in current_docIds): 
                current_postings_list[current_docIds.index(posting.doc_id)]+=posting.payload 
            else:
                current_postings_list.append(posting)
        
    def __write_term_posting_list_to_disk(self,file_name:str,term:str,merged_postings_list:list,split_results_in_files:bool):
        """ This method is used to write in a file on disk in append mode a full entry of the lexicon in the format ex.
            term doc_id_1:term_freq_1 doc_id_2:term_freq_2 doc_id_3:term_freq_3 ... 
            
            Args:
                file_name: the name of the output file 
                term: the lexicon term
                merged_postings_list: the full merged posting list
                split_results_in_files: Specify if you want or not the inderted index in two different files: the doc_ids file and the term_freq file
        """
       
        
        if(not split_results_in_files):
            with open(file_name+self.OUTPUT_FILE_FORMAT, "a") as f:
                f.write(term)
                for posting in merged_postings_list:
                    f.write(f" {posting.doc_id}")
                    if posting.payload:
                        f.write(f":{str(posting.payload)}")
                f.write("\n")
        else:
            
             with open(file_name+"_"+self.NAME_DOC_IDS_FILE+self.OUTPUT_FILE_FORMAT, "a") as f:
                for index, posting in enumerate(merged_postings_list):
                    f.write(str(posting.doc_id))
                    if index != len(merged_postings_list) - 1:
                        f.write(",")
                f.write("\n")
            
             with open(file_name+"_"+self.NAME_TERM_FREQ_FILE+self.OUTPUT_FILE_FORMAT, "a") as f:
                for index, posting in enumerate(merged_postings_list):
                    f.write(str(posting.payload))
                    if index != len(merged_postings_list) - 1:
                        f.write(",")
                f.write("\n")

# Example of usage

In [4]:
# tot_doc=[
#             "The pen is on the table",
#             "The day is very sunny",
#             "Goodmoring new article",
#             "A cat is faster then a dog",
#             "How are you",
#             "A boy is a man with low age",
#             "Lake Ontario is one of the biggest lake in the world",
#             "English is worst than Italian",
#             "Spiderman is the best superhero in Marvel universe",
#             "Last night I saw a Netflix series",
#             "A penny for your thoughts",
#             "Actions speak louder than words",
#             "All that glitters is not gold",
#             "Beauty is in the eye of the beholder",
#             "Birds of a feather flock together",
#             "Cleanliness is next to godliness",
#             "Don't count your chickens before they hatch",
#             "Every people cloud has a silver lining people",
#             "Fool me once shame on you fool me twice shame on me",
#             "Honesty is the best policy.",
#             "If the shoe fits, wear it",
#             "It's a piece of cake",
#             "Jump on the bandwagon",
#             "Keep your chin up",
#             "Let the cat out of the bag",
#             "Make a long story short",
#             "Necessity is the mother of invention",
#             "Once in a blue moon",
#             "Practice makes perfect",
#             "Read between the lines",
#             "The early bird catches people the worm",
#             "The pen is mightier than the sword",
#             "There's no smoke without fire",
#             "To each his own",
#             "Two heads are better than one",
#             "You can't have your cake and eat it too",
#             "A watched pot never boils",
#             "Beggars can't be choosers",
#             "Better late than never",
#             "Calm before the storm",
#             "Curiosity killed the cat",
#             "Every dog has its day",
#             "Great minds think alike",
#             "Hope for the best prepare for the worst",
#             "Ignorance is bliss.",
#             "It's the last straw that breaks the camel's back",
#             "Laugh and the world laughs with you weep and you weep alone",
#             "Money can't buy happiness",
#             "No news is good news",
#             "Out of sight out of mind",
#             "People who live in glass houses shouldn't throw stones",
#             "Rome wasn't built in a day",
#             "Silence is golden",
#             "The apple doesn't fall far from the tree",
#             "The more, the merrier",
#             "There's no place like home",
#             "Two wrongs don't make a right",
#             "When in Rome do as the Romans do",
#             "You reap what you sow",
#             "People people people"]


# indexBuilder=IndexBuilder()
# #ii=indexBuilder.build_in_memory_index(tot_doc)
# indexBuilder.build_block_sort_base_indexing(tot_doc,"complete_inverted_index",2220,False,True)


In [5]:
tot_doc=[
    "0     The pen is on the table",
    "1     The day is very sunny",
    "2     Goodmoring new article",
    "3     A cat is faster then a dog",
    "4     How are you",
    "5     A boy is a man with low age",
    "6     Lake Ontario is one of the biggest lake in the world",
    "7     English is worst than Italian",
    "8     Spiderman is the best superhero in Marvel universe",
    "9     Last night I saw a Netflix series",
    "10    A penny for your thoughts",
    "11    Actions speak louder than words",
    "12    All that glitters is not gold",
    "13    Beauty is in the eye of the beholder",
    "14    Birds of a feather flock together",
    "15    Cleanliness is next to godliness",
    "16    Don't count your chickens before they hatch",
    "17    Every people cloud has a silver lining people",
    "18    Fool me once shame on you fool me twice shame on me",
    "19    Honesty is the best policy.",
    "20    If the shoe fits, wear it",
    "21    It's a piece of cake",
    "22    Jump on the bandwagon",
    "23    Keep your chin up",
    "24    Let the cat out of the bag",
    "25    Make a long story short",
    "26    Necessity is the mother of invention",
    "27    Once in a blue moon",
    "28    Practice makes perfect",
    "29    Read between the lines",
    "30    The early bird catches people the worm",
    "31    The pen is mightier than the sword",
    "32    There's no smoke without fire",
    "33    To each his own",
    "34    Two heads are better than one",
    "35    You can't have your cake and eat it too",
    "36    A watched pot never boils",
    "37    Beggars can't be choosers",
    "38    Better late than never",
    "39    Calm before the storm",
    "40    Curiosity killed the cat",
    "41    Every dog has its day",
    "42    Great minds think alike",
    "43    Hope for the best prepare for the worst",
    "44    Ignorance is bliss.",
    "45    It's the last straw that breaks the camel's back",
    "46    Laugh and the world laughs with you weep and you weep alone",
    "47    Money can't buy happiness",
    "48    No news is good news",
    "49    Out of sight out of mind",
    "50    People who live in glass houses shouldn't throw stones",
    "51    Rome wasn't built in a day",
    "52    Silence is golden",
    "53    The apple doesn't fall far from the tree",
    "54    The more, the merrier",
    "55    There's no place like home",
    "56    Two wrongs don't make a right",
    "57    When in Rome do as the Romans do",
    "58    You reap what you sow",
    "59    People people people"
]


indexBuilder=IndexBuilder()
indexBuilder.build_block_sort_base_indexing(tot_doc,"complete_inverted_index",2220,False,True)

Index Builder costructor
